In [12]:
import os
import json
import time
import requests
import psycopg2
from datetime import datetime, timezone
from psycopg2.extras import execute_values
from ipyleaflet import (
    Map, LayerGroup, Marker, DivIcon, FullScreenControl,
    basemaps, TileLayer, GeoJSON, WidgetControl
)
from ipywidgets import (
    Layout, SelectionSlider, VBox, HTML, IntRangeSlider, Checkbox, Play, jslink,
    FloatRangeSlider, HBox, Label, VBox as WidgetVBox
)

USERNAME = os.getenv("OPENSKY_USERNAME", None)
PASSWORD = os.getenv("OPENSKY_PASSWORD", None)

BLUE = "#28a745"
GREEN = "#007bff"

MAX_PLANES = 100

DB_CONFIG = {
    "host": "localhost",
    "port": 15000,
    "database": "flight_data",
    "user": "admin",
    "password": "admin"
}

plane_svg = ('<div class="plane-label">'
             '{callsign} </div>'
             '<svg width="24px" height="24px" viewBox="0 0 512 512" style="transform:rotate({heading}deg); transition: transform 0.2s linear; border: none;"><path fill="{color}" d="M448 336v-40L288 192V79.2c0-22.1-17.9-40-40-40s-40 17.9-40 40V192L48 296v40l160-48v113.6l-48 31.2V472l88-24 88 24v-39.2l-48-31.2V288l160 48z" stroke="white"/></svg>')

style_html = HTML("""
<style>
    /* 1. Hides the label by default */
    .plane-label {
        display: none;
    }

    /* 2. Shows the label when you hover over the icon container */
    /* We target the class 'plane-icon' which you assign in Python */
    .plane-icon:hover .plane-label {
        border: none;
        width: 60px;'
        padding-up: 3px;'
        background-color: gray;'
        color: orange;'
        border-radius: 5px;'
        font-weight: bold;'
        text-align: center;
    }
</style>
""")

def initialize_database():
    # 1. Connect to 'postgres' database to check/create 'flight_data'
    temp_config = DB_CONFIG.copy()
    temp_config["database"] = "postgres"
    
    try:
        conn = psycopg2.connect(**temp_config)
        conn.set_session(autocommit=True)
        cur = conn.cursor()
        
        # Check if database exists
        cur.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = 'flight_data';")
        exists = cur.fetchone()
        if not exists:
            print("Creating database flight_data...")
            cur.execute("CREATE DATABASE flight_data;")
        
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error checking/creating database: {e}")

    # 2. Now connect to 'flight_data' to initialize extensions and tables
    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    print("Initializing Database...")
    cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS timescaledb;")
    cur.execute("""
    CREATE TABLE IF NOT EXISTS observations (
        time        TIMESTAMPTZ NOT NULL,
        icao24      VARCHAR(10) NOT NULL,
        callsign    VARCHAR(20),
        location    GEOMETRY(POINT, 4326),
        altitude    DOUBLE PRECISION,
        velocity    DOUBLE PRECISION,
        heading     DOUBLE PRECISION,
        on_ground   BOOLEAN
    );
    """)
    try:
        cur.execute("SELECT create_hypertable('observations', 'time');")
        print("Hypertable created successfully.")
    except Exception as e:
        if "already exists" in str(e):
             print("Hypertable already exists, skipping.")
        else:
             print(f"Hypertable creation skipped or failed: {e}")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_icao_time ON observations (icao24, time DESC);")

    cur.close()
    conn.close()
    print("Database is ready!")


# Delete all the rows from the database.
def clear_the_database():
    temp_config = DB_CONFIG.copy()
    temp_config["database"] = "postgres"

    conn = psycopg2.connect(**DB_CONFIG)
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    try:
        cur.execute("DELETE FROM observations;")
    except Exception as e:
        print(f"Error while clearing the  database. {e}")

    cur.close()
    conn.close()
    print("Database is cleaned!")

def check_row_count():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM observations;")
    count = cur.fetchone()[0]
    print(f"Current rows in DB: {count}")
    cur.close()
    conn.close()    

def save_to_db(planes):
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    execute_values(cur, """
        INSERT INTO observations (time, icao24, callsign, location, altitude, velocity, heading, on_ground)
        VALUES %s""",
        [(datetime.fromtimestamp(p['time'], tz=timezone.utc), p['icao24'], p['callsign'], f"SRID=4326;POINT({p['lon']} {p['lat']})",
          p['altitude'], p['velocity'], p['heading'], p['on_ground']) for p in planes])

    conn.commit()
    cur.close()
    conn.close()
    print(f"Successfully saved {len(planes)} rows to TimescaleDB.")
    
def get_live_planes():
    global USERNAME, PASSWORD
    opensky_url = "https://opensky-network.org/api/states/all"
    auth = (USERNAME, PASSWORD) if USERNAME and PASSWORD else None

    try:
        response = requests.get(opensky_url, auth=auth, timeout=10)
        response.raise_for_status()
        data = response.json()

        states = data.get('states', [])
        timestamp = data.get('time')

        processed_planes = []

        for s in states:
            if s[5] is not None and s[6] is not None:
                processed_planes.append({
                    "time": timestamp,
                    "icao24": s[0],
                    "callsign": s[1].strip() if s[1] else "NONE",
                    "lon": s[5],
                    "lat": s[6],
                    "altitude": s[7], # Barometric altitude
                    "velocity": s[9],
                    "heading": s[10] or 0, # Default to 0 if null
                    "on_ground": s[8]
                })

        return processed_planes

    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

# Get 40 timestamps of planes from the API
def import_planes_from_opensky():
    for i in range(40):
        planes = get_live_planes()
        save_to_db(planes)
        time.sleep(2)
        print(f"Imported {len(planes)} planes from OpenSky Network.")
    check_row_count()
    
# Coutnry Borders
def create_country_geojson():       
    url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
    geo_json_data = requests.get(url).json()
    
    style = {
        'color': '#666',
        'fillColor': '#333',
        'opacity': 1,
        'fillOpacity': 0.1,
        'weight': 1
    }
    
    hover_style = {
        'fillColor': '#007bff',
        'fillOpacity': 0.5
    }
    
    countries_layer = GeoJSON(
        data=geo_json_data,
        style=style,
        hover_style=hover_style,
        name="Countries"
    )
    
    selected_style = {
        'fillColor': 'red',
        'fillOpacity': 0.5,
        'color': 'white',
        'weight': 2
    }
    return countries_layer, selected_style

def get_all_timestamps():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        cur.execute("SELECT DISTINCT time FROM observations ORDER BY time ASC")
        rows = cur.fetchall()
        conn.close()
        return [r[0] for r in rows]
    except Exception as e:
        print(f"Error fetching timestamps: {e}")
        return []

# Clear the database if you want a "fresh start" with newly imported planes
# clear_the_database()

# Fetch planes from API (uncomment only once)
# import_planes_from_opensky()

print(get_all_timestamps())

[datetime.datetime(2026, 1, 17, 11, 11, 17, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 11, 23, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 11, 38, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 11, 41, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 11, 45, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 11, 48, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 2, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 4, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 5, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 15, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 18, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 22, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 32, tzinfo=datetime.timezone.utc), datetime.datetime(2026, 1, 17, 11, 12, 3

In [13]:
check_row_count()


Current rows in DB: 212144


In [14]:
# Static map movement
m = Map( # Center on London [51.5074, -0.1278]
    center=(51.5, -0.1),
    zoom=7,
    scroll_wheel_zoom=True,
    world_copy_jump=False,
    basemap=basemaps.CartoDB.Positron,
    layout=Layout(width='100%', height='600px')
)

# hard limits so you can't pan into the "gray void" past the North/South poles
m.max_bounds = [(-90, -180), (90, 180)]

# import_planes_from_opensky()

plane_layer = LayerGroup()
m.add_layer(plane_layer)
m.add_control(FullScreenControl())

def get_visible_planes(south, west, north, east):
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()

        query = f"""
        SELECT DISTINCT ON (icao24)
               icao24, callsign, ST_Y(location::geometry) as lat, ST_X(location::geometry) as lon,
               heading, on_ground
        FROM observations
        WHERE location && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
        LIMIT {MAX_PLANES};
        """

        cur.execute(query, (west, south, east, north))
        data = cur.fetchall()
        cur.close()
        conn.close()
        return data
    except Exception as e:
        print(f"DB Error: {e}")
        return []

def update_view(change=None):
    bounds = m.bounds
    if not bounds or len(bounds) < 2:
        south, west, north, east = 49.0, -2.0, 53.0, 2.0
    else:
        (south, west), (north, east) = bounds

    planes = get_visible_planes(south, west, north, east)
    print(f"found {len(planes)} planes with GPS data.")

    new_markers = []
    for p in planes[:MAX_PLANES]:
        icao, callsign, lat, lon, heading, on_ground = p

        color = BLUE if on_ground else GREEN
        icon_html = plane_svg.format(heading=heading or 0, color=color, callsign=callsign)
        icon = DivIcon(html=icon_html, icon_size=[0, 0], class_name="plane-icon")

        new_markers.append(Marker(location=(lat, lon), icon=icon, title=callsign, draggable=False))

    plane_layer.clear_layers()
    plane_layer.layers = tuple(new_markers)

m.observe(update_view, names='bounds')
# display(m)

In [15]:
class TimeSliderTracker:
    def __init__(self, m, layer_group, slider_widget):
        self.map = m
        self.layer_group = layer_group
        self.slider = slider_widget
        self.active_markers = {}
        self.current_bounds = ()
        self.conn = None
        self.selected_country_geom = None
        self.selected_country_id = None

        self.moving_map = True

        self.altitude_range = (0, 20000)
        self.velocity_range = (0, 1500)
        self.svg_template = plane_svg

    def connect_db(self):
        if self.conn is None or self.conn.closed != 0:
            try:
                self.conn = psycopg2.connect(**DB_CONFIG)
            except Exception as e:
                print(f"DB Connection Error: {e}")

    def generate_html_string(self, icao, callsign, lat, lon, heading, altitude, velocity, on_ground):
        status = "On Ground" if on_ground else "In Flight"
        status_color = "blue" if on_ground else "green"

        altitude_str = f"{altitude:.0f} m" if altitude else "N/A"
        velocity_str = f"{velocity:.1f} m/s" if velocity else "N/A"
        heading_str = f"{heading:.0f}°" if heading else "N/A"

        return f"""
        <div style="font-family: Arial, sans-serif; min-width: 200px;">
            <h4 style="margin: 0 0 10px 0; color: {status_color};">{callsign}</h4>
            <table style="width: 100%; font-size: 12px;">
                <tr><td><b>ICAO24:</b></td><td>{icao}</td></tr>
                <tr><td><b>Status:</b></td><td style="color: {status_color};">{status}</td></tr>
                <tr><td><b>Latitude:</b></td><td>{lat:.4f}</td></tr>
                <tr><td><b>Longitude:</b></td><td>{lon:.4f}</td></tr>
                <tr><td><b>Altitude:</b></td><td>{altitude_str}</td></tr>
                <tr><td><b>Velocity:</b></td><td>{velocity_str}</td></tr>
                <tr><td><b>Heading:</b></td><td>{heading_str}</td></tr>
            </table>
        </div>
        """

    def get_planes(self, timestamp):
        # Bounds centered around West-Europe
        if not self.current_bounds:
            south, west, north, east = 35.0, -10.0, 60.0, 40.0
        else:
            ((south, west), (north, east)) = self.current_bounds

        try:
            self.connect_db()
            with self.conn.cursor() as cur:
                query = """
                SELECT icao24, callsign, ST_Y(location::geometry), ST_X(location::geometry), heading, on_ground, altitude, velocity
                FROM observations
                WHERE time = %s
                AND location && ST_MakeEnvelope(%s, %s, %s, %s, 4326)
                AND altitude BETWEEN %s AND %s
                AND velocity BETWEEN %s AND %s
                """
                params = [
                    timestamp, west, south, east, north,
                    self.altitude_range[0], self.altitude_range[1],
                    self.velocity_range[0], self.velocity_range[1]
                ]

                if self.selected_country_geom:
                    query += " AND ST_Intersects(location, ST_GeomFromGeoJSON(%s))"
                    params.append(json.dumps(self.selected_country_geom))

                query += f" LIMIT {MAX_PLANES};"

                cur.execute(query, tuple(params))
                return cur.fetchall()

        except Exception as e:
            print(f"Query Error: {e}")
            return []

    def update_frame(self, timestamp):
        planes = self.get_planes(timestamp)
        current_icaos = set()

        for p in planes:
            icao, callsign, lat, lon, heading, on_ground, altitude, velocity = p
            current_icaos.add(icao)

            lat, lon = round(lat, 4), round(lon, 4)
            heading = int(heading) if heading else 0

            # Generate the HTML string content for pop-ups.
            new_popup_html = self.generate_html_string(icao, callsign, lat, lon, heading, altitude, velocity, on_ground)

            # If marker already exists, then only update its position. No need for recreating it.
            if icao in self.active_markers:
                marker = self.active_markers[icao]

                # Update the position of the plane. This takes into considerations only planes that were already displayed in the current window.
                if marker.location != (lat, lon):
                    marker.location = (lat, lon)

                # Update the heading degrees, hence the plane's compass angle only the difference is noticeable (in this case > 5 degrees comparing to previous timestamp state).
                if abs(marker.last_heading - heading) > 5:
                    color = "#28a745" if on_ground else "#007bff"
                    icon_html = self.svg_template.format(heading=heading, color=color, callsign=callsign)

                    # Need to create a new icon with the modified angle.
                    marker.icon = DivIcon(
                        html=icon_html,
                        icon_size=[0, 0],
                        class_name="plane-icon"
                    )
                    marker.last_heading = heading

                # Update the marker's pop-up value
                if marker.popup:
                    marker.popup.value = new_popup_html

            # If the plane was not populating the map already, then create a new marker to represent the new plane
            else:
                color = "#28a745" if on_ground else "#007bff"
                icon_html = self.svg_template.format(heading=heading, color=color, callsign=callsign)

                new_icon = DivIcon(
                    html=icon_html,
                    icon_size=[0, 0],
                    border=None,
                    class_name="plane-icon"
                )

                popup_widget = HTML(value=new_popup_html)

                marker = Marker(location=(lat, lon), icon=new_icon, draggable=False, title=callsign)
                marker.last_heading = heading
                marker.popup = popup_widget

                # Add the newly created plane marker to the existing ones so if the next timestamp state includes the plane, a new marker will not be created.
                self.active_markers[icao] = marker

        # Remove the markers that are outside the scope of the focused map area. No need to keep them.
        keys_to_remove = [k for k in self.active_markers if k not in current_icaos]
        for k in keys_to_remove:
            if self.active_markers[k].popup:
                self.active_markers[k].popup.close()
            del self.active_markers[k]

        self.layer_group.layers = tuple(self.active_markers.values())

    def update_bounds(self, change):
        self.current_bounds = change['new']

        if self.moving_map:
            current_time = self.slider.value
            self.update_frame(current_time)

timestamps = get_all_timestamps()


In [16]:
# Run the previous code snippets before this one.

if not timestamps:
    print("No data found in DB!")
else:
    m = Map(
        center=(50, 10), zoom=6,
        basemap=basemaps.CartoDB.Positron,
        layout=Layout(width='100%', height='1000px')
    )
    plane_layer = LayerGroup()
    countries_layer, selected_style = create_country_geojson()
    m.add_layer(plane_layer)
    m.add_layer(countries_layer)

    # Timestamps slider
    slider = SelectionSlider(
        options=timestamps,
        description='Time:',
        continuous_update=False,
        orientation='horizontal',
        layout=Layout(width='90%')
    )

    # Plane Tracker setup
    tracker = TimeSliderTracker(m, plane_layer, slider)
    # Set the callback function when map bound values are changed. (Map position changed)
    m.observe(tracker.update_bounds, names='bounds')


    time_label = HTML(value=f"<b>Selected Time:</b> {timestamps[0]}")

    # Altitude and plane velocity filters.
    altitude_slider = IntRangeSlider(
        value=[0, 20000], min=0, max=20000, step=500,
        description='Altitude:', continuous_update=False, layout=Layout(width='400px')
    )
    velocity_slider = IntRangeSlider(
        value=[0, 1500], min=0, max=1500, step=50,
        description='Velocity(m/s):', continuous_update=False, layout=Layout(width='380px')
    )

    # Autonomous planes movement.
    play = Play(
        value=0,
        min=0,
        max=len(timestamps) - 1,
        step=1,
        interval=1000,
        description="Auto-Play",
        show_repeat=False
    )

    # Link the Play widget (value) to the Slider (index)
    jslink((play, 'value'), (slider, 'index'))

    # Filter callback by altitude and velocity
    def on_filter_change(change):
        tracker.altitude_range = altitude_slider.value
        tracker.velocity_range = velocity_slider.value
        tracker.update_frame(slider.value)

    # Set observers when user changes the filter values
    altitude_slider.observe(on_filter_change, names='value')
    velocity_slider.observe(on_filter_change, names='value')

    play_container = HBox([play])
    play_container.layout.justify_content = 'center'
    play_container.layout.margin = '10px 0 0 0'

    # Altitude and plane velocity filters
    filter_container = WidgetVBox([
        HTML("<div style='text-align:center;'><b>Filtre Avioane</b> </div>"),
        altitude_slider,
        velocity_slider,
        HTML("<div style='text-align:center; margin-top:10px;'> <b>Animation Start/Stop</b> </div>"),
        play_container
    ])
    filter_container.layout.padding = '20px'
    filter_container.layout.background = 'rgba(40, 40, 40, 0.3)'
    filter_container.layout.border = '2px solid gray'
    filter_container.layout.border_radius = '5px'

    filter_control = WidgetControl(widget=filter_container, position='topright')
    m.add_control(filter_control)

    # Select a specific country air-space planes.
    selected_country_layer = GeoJSON(data={'type': 'FeatureCollection', 'features': []}, style=selected_style)
    m.add_layer(selected_country_layer)

    def on_country_click(feature, **kwargs):
        country_id = feature.get('id') or feature['properties'].get('name')
        if tracker.selected_country_id == country_id:
            tracker.selected_country_id = None
            tracker.selected_country_geom = None
            selected_country_layer.data = {'type': 'FeatureCollection', 'features': []}
        else:
            tracker.selected_country_id = country_id
            tracker.selected_country_geom = feature['geometry']
            selected_country_layer.data = {'type': 'FeatureCollection', 'features': [feature]}
        tracker.update_frame(slider.value)

    countries_layer.on_click(on_country_click)
    selected_country_layer.on_click(on_country_click)

    # Manually change the timestamps.
    def on_slider_change(change):
        ts = change['new']
        time_label.value = f"<b>Selected Time:</b> {ts}"
        tracker.update_frame(ts)

    slider.observe(on_slider_change, names='value')

    # Initialize first frame
    on_slider_change({'new': timestamps[0]})

    ui = VBox([style_html, m, slider, time_label])
    display(ui)